In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

### COULD NOT MAKE IT WORK WITH DATA FROM THE REPO FOLDER 

RECEIVED 2D ARRAY, EXPECTED 1D (COULDN'T FIGURE IT OUT, even using the 'reshape' function)

In [ ]:
num_df = pd.read_csv('numerical.csv')
cat_df = pd.read_csv('categorical.csv')
target_df = pd.read_csv('target.csv')

In [ ]:
cat_columns = cat_df.columns
one_hot = OneHotEncoder()
cat_encoded = one_hot.fit_transform(cat_df)
array = cat_encoded.toarray()
df_cat_encoded = pd.DataFrame(array, columns=one_hot.get_feature_names_out(cat_columns))
df_cat_encoded.head()


In [ ]:
data = pd.concat([num_df, df_cat_encoded,target_df], axis=1)

In [ ]:
transformer = MinMaxScaler().fit(data)
data_norm = transformer.transform(data)
data_norm = pd.DataFrame(data_norm, columns=data.columns)
data_norm.head()

In [ ]:
X = data_norm.drop(['TARGET_B', 'TARGET_D'], axis=1)
y = data_norm['TARGET_B']

In [ ]:
X_upsample, y_upsample = SMOTE().fit_resample(X,y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_upsample, y_upsample, test_size=0.3, random_state=100)


#testing Random Forest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

#predict
predictions = rf.predict(X_test)

#results
accuracy = rf.score(y_test, predictions)
accuracy

### USE CHURN DATA FROM PREVIOUS LAB INSTEAD

In [2]:
data = pd.read_csv('Customer-Churn.csv')

In [3]:
data.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [4]:
def replace_empty(data):
    for columns in data.columns:
        data[columns] = data[columns].replace(' ', None)
        data[columns] = data[columns].replace('', None)

replace_empty(data)

In [5]:
data['TotalCharges']= data['TotalCharges'].astype(float)

data['SeniorCitizen']= data['SeniorCitizen'].astype(object)
data['SeniorCitizen'] = data['SeniorCitizen'].replace(0, 'No')
data['SeniorCitizen'] = data['SeniorCitizen'].replace(1, 'Yes')

In [6]:
num = data.select_dtypes(np.number)
cat = data.select_dtypes(np.object_)

#encode categoricals
cat_dum = pd.get_dummies(cat, drop_first=True)

In [7]:
print(num.shape)
print(cat_dum.shape)

(7043, 3)
(7043, 20)


In [8]:
#normalize and transform the data
transformer = MinMaxScaler().fit(num)
num_norm = transformer.transform(num)
num_norm = pd.DataFrame(num_norm, columns=num.columns)
num_norm.head()

,tenure,MonthlyCharges,TotalCharges
0,0.013889,0.115423,0.001275
1,0.472222,0.385075,0.215867
2,0.027778,0.354229,0.010310
3,0.625000,0.239303,0.210241
4,0.027778,0.521891,0.015330


In [9]:
df_final = pd.concat([num_norm, cat_dum], axis=1)

In [10]:
df_final.drop_duplicates()
df_final.dropna() #after removing duplicates and nan values we receive the error 'input contains infinity or values too large for dtype'float64'

,tenure,MonthlyCharges,TotalCharges,gender_Male,SeniorCitizen_Yes,Partner_Yes,Dependents_Yes,PhoneService_Yes,OnlineSecurity_No internet service,OnlineSecurity_Yes,...,DeviceProtection_Yes,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,Churn_Yes
0,0.013889,0.115423,0.001275,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.472222,0.385075,0.215867,1,0,0,0,1,0,1,...,1,0,0,0,0,0,0,1,0,0
2,0.027778,0.354229,0.010310,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
3,0.625000,0.239303,0.210241,1,0,0,0,0,0,1,...,1,0,1,0,0,0,0,1,0,0
4,0.027778,0.521891,0.015330,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0.333333,0.662189,0.227521,1,0,1,1,1,0,1,...,1,0,1,0,1,0,1,1,0,0
7039,1.000000,0.845274,0.847461,0,0,1,1,1,0,0,...,1,0,0,0,1,0,1,1,0,0
7040,0.152778,0.112935,0.037809,0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7041,0.055556,0.558706,0.033210,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [15]:
#we must remove these large values, we can consider them to be outliers 
df_final = df_final[np.isfinite(df_final).all(1)] 
df_final

,tenure,MonthlyCharges,TotalCharges,gender_Male,SeniorCitizen_Yes,Partner_Yes,Dependents_Yes,PhoneService_Yes,OnlineSecurity_No internet service,OnlineSecurity_Yes,...,DeviceProtection_Yes,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,Churn_Yes
0,0.013889,0.115423,0.001275,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.472222,0.385075,0.215867,1,0,0,0,1,0,1,...,1,0,0,0,0,0,0,1,0,0
2,0.027778,0.354229,0.010310,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
3,0.625000,0.239303,0.210241,1,0,0,0,0,0,1,...,1,0,1,0,0,0,0,1,0,0
4,0.027778,0.521891,0.015330,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0.333333,0.662189,0.227521,1,0,1,1,1,0,1,...,1,0,1,0,1,0,1,1,0,0
7039,1.000000,0.845274,0.847461,0,0,1,1,1,0,0,...,1,0,0,0,1,0,1,1,0,0
7040,0.152778,0.112935,0.037809,0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7041,0.055556,0.558706,0.033210,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [16]:
y = df_final['Churn_Yes']
X = df_final.drop(['Churn_Yes'], axis=1)

In [17]:
print(X.shape)
print(y.shape)

(7032, 22)
(7032,)


This is the test with no changes

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [20]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

#predict test data
y_pred_1 = log_reg.predict(X_test)

#create model
log_accuracy = log_reg.score(X_test, y_test)

#results
print("LogReg Accuracy:", log_accuracy)

LogReg Accuracy: 0.7938388625592417


the log reg model has a pretty good accuracy score of 0.79

Let's see when we upsamble with SMOTE

In [21]:
X_upsample, y_upsample = SMOTE().fit_resample(X,y)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_upsample, y_upsample, test_size=0.3, random_state=100)

In [23]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

#predict test data
y_pred_1 = log_reg.predict(X_test)

#create model
log_accuracy = log_reg.score(X_test, y_test)

#results
print("LogReg Accuracy:", log_accuracy)

LogReg Accuracy: 0.7850225952227243


The score has decreased a bit. The model was better before upsampling the data...

Let's see if we can identify the best parameters and make the accuracy score even better after using SMOTE

In [24]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100], 
    'min_samples_split': [2, 4], 
    'min_samples_leaf' : [1, 2,5],
    'max_features': ['sqrt','log2']
    } #set paramaters for the model - here we are making a different test with different sample sizes, features etc for every parameter

clf = RandomForestClassifier(random_state=100)

grid_search = GridSearchCV(clf, param_grid, cv=5,return_train_score=True)
grid_search.fit(X_train,y_train)
grid_search.best_params_ #To check the best set of parameters returned

{'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 100}

In [25]:
#apply the recommended features 

from sklearn.model_selection import cross_val_score


clf = RandomForestClassifier(random_state=0, max_features='sqrt', min_samples_leaf=1, min_samples_split=4, n_estimators=100)
cross_val_scores = cross_val_score(clf, X_train, y_train, cv=5)
print(round(np.mean(cross_val_scores),3))

0.816


The model has improved and is more accruate after upsampling and applying the correct parameters to the RandomForest model